In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import numpy as np
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
embedding = OpenAIEmbeddings(model = 'text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000013B3C4B1110>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000013B3C4D0890>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [4]:
text = "This is my first tutorial of OpenAI embedding"
embedded_sentence = embedding.embed_query(text)
embedded_sentence

[-0.006216730922460556,
 -0.01371654961258173,
 0.0317251980304718,
 -0.019938956946134567,
 -0.002668368397280574,
 -0.02597969025373459,
 0.005328220780938864,
 0.014204803854227066,
 -0.011854369193315506,
 0.01477254182100296,
 0.05581999570131302,
 -0.05527496710419655,
 -0.009634513407945633,
 -0.057727597653865814,
 0.015783116221427917,
 -0.006415438838303089,
 -0.006795823574066162,
 -0.01440919004380703,
 0.000347739493008703,
 0.027932707220315933,
 0.02253919653594494,
 -0.02165352553129196,
 -0.03599458560347557,
 0.05027887597680092,
 0.015544665977358818,
 -0.06117944419384003,
 0.05722798779606819,
 0.0496884286403656,
 -0.011990626342594624,
 -0.0308168176561594,
 0.012910361401736736,
 -0.014647639356553555,
 -0.01862180605530739,
 -0.030498884618282318,
 -0.014965573325753212,
 0.017384136095643044,
 0.007284078281372786,
 0.022516487166285515,
 -0.03299693018198013,
 0.030612431466579437,
 0.03224751725792885,
 -0.019882183521986008,
 -0.013807387091219425,
 0.01966

In [5]:
np.array(embedded_sentence).shape

(1536,)

In [6]:
embedding_512 = OpenAIEmbeddings(model = 'text-embedding-3-small', dimensions = 512)
embedded_sentence_512 = embedding_512.embed_query(text)
np.array(embedded_sentence_512).shape

(512,)

In [7]:
loader = TextLoader('speech.txt')
doc = loader.load()
doc

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 512, chunk_overlap = 64)
chunked_doc = text_splitter.split_documents(doc)
chunked_doc

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_co

#### Vector Embedding and Vector StoreDB

In [9]:
db = Chroma.from_documents(chunked_doc,embedding)
db

In [10]:
query = "We have borne with their present government through all these bitter months because of that friendship—exercising"
search_results = db.similarity_search(query)
print(search_results)

[Document(metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, still have an opportunity to prove that friendship in our daily attitude and actions toward the millions of men and women of German birth and native sympathy who live among us and share our life, and we shall be proud to prove it toward all who are in fact loyal to their neighbors and to the government in'), Document(metadata={'source': 'speech.txt'}, page_content='It will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is